In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#first we read in the dataset
netflix=pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv', parse_dates=['date_added'])
netflix.head()

In [ ]:
netflix.shape

In [ ]:
netflix.columns

In [ ]:
netflix.isnull().sum()

In [ ]:
netflix.isnull().sum(axis=1).sort_values(ascending = False)

In [ ]:
netflix.title.nunique()

In [ ]:
netflix.title.unique()


In [ ]:
n = netflix.drop(netflix[netflix.cast.isnull() == True].index).copy()

In [ ]:
n.drop(n[n.country.isnull() == True].index, inplace = True)

In [ ]:
n.shape

# Let's look for duplicates****

In [ ]:
n.title.value_counts()

# **>>*Certain titles aren't formulated in English!***

In [ ]:
n.title.sort_values()

In [ ]:
n.loc[3799]

In [ ]:
n.country.value_counts()

In [ ]:
#n[n.country.str.contains('France')].count()

In [ ]:
n[n.country == 'South Korea'].count()

In [ ]:
n1 = n.set_index('title', inplace=False).copy()

In [ ]:
n1.loc['Tunnel']

In [ ]:
n1.loc['Love']

In [ ]:
n1.loc['Limitless']

In [ ]:
n1.loc['Oh My Ghost']

In [ ]:
n1.loc["The Silence"]

# **So: various name duplicates, but it seems refering to totally different movies!**

# Now let's find the movies which are real duplicates****

In [ ]:
n.groupby(["title",'director']).filter(lambda movies_same_title: len(movies_same_title)>1).index

In [ ]:
n.loc[[2123, 2124, 2387, 2388]]

# **We can remove this movie dupplicate "Sarkar" from the dataset**

In [ ]:
n.loc[2123]

In [ ]:
n.drop(index=2124, inplace=True)

In [ ]:
n.shape

In [ ]:
import matplotlib.pyplot as plt

- **What is the proportion of TV Series vs Movies?
- How many actors are represented?**
- Genre most represented per country (excl. US)? 
- Which are the most represented actors per country? 
- What is the average time for a movie to reach Netflix catalogue?
- Prefered genres per country? Evolution of genre represented as an indication of viewers' preferences change? 

# TV Series vs Movies****

In [ ]:
n.groupby('release_year').type.value_counts(normalize = True).unstack().plot()

In [ ]:
ser_m = n.groupby('release_year').type.value_counts(normalize = True).unstack().Movie.copy() * 100
ser_tv = n.groupby('release_year').type.value_counts(normalize = True).unstack()['TV Show'].copy() * 100

In [ ]:
ser_tv

In [ ]:
ser_m

In [ ]:
ser_m.fillna(0, inplace = True)
ser_tv.fillna(0, inplace = True)

In [ ]:
n.groupby('release_year').type.value_counts(normalize=True).unstack().plot.bar(figsize = (9, 7), stacked = True)

In [ ]:
# fig = plt.figure(figsize=(12, 9))
# ax = fig.gca()
# ax.bar(x = np.arange(len(ser_m)), height=ser_m,
#        align='center', color='g'
#       )
# ax.bar(x = np.arange(len(ser_m)),height=ser_tv, 
#        bottom=ser_m, 
#        align='center', color='r'
#       )
# #ax.set_xlim([-1, 7])
# #ax.legend(['Buyers', 'Non-buyers'], loc='best')
# #ax.set_title('Bar chart of groups per day')
# ax.set_ylabel('Proportion TV Show vs Movies')
# plt.show()

# How many actors are represented? Actors most represented?****

In [ ]:
n.columns

In [ ]:
n1.cast.str.split(",", expand=True).stack().nunique()#.value_counts()#.plot.bar(figsize=(10, 7))

In [ ]:
n.cast.str.split(",", expand=True).stack().reset_index

In [ ]:
n.cast.str.split(",", expand=True).stack().reset_index(1).drop('level_1', axis=1).rename({0: 'actors'}, axis=1)#.value_counts()#.plot.bar(figsize=(10, 7))

In [ ]:
act = n.cast.str.split(",", expand=True).stack().reset_index(1).drop('level_1', axis=1).rename({0: 'actors'}, axis=1).copy()

In [ ]:
net2 = n.merge(act, right_index = True, left_index=True).copy()

In [ ]:
net2.head()

In [ ]:
# net2.groupby("actors").title.count().plot(figsize = (15, 15))#.apply(lambda titles: titles.tolist())

In [ ]:
net2.groupby("actors").title.apply(lambda titles: titles.tolist())

In [ ]:
net2.groupby('actors').title.count().max()

In [ ]:
net2.groupby('actors').title.count().idxmax()

In [ ]:
net2.groupby("actors").title.apply(lambda titles: titles.tolist())[' Anupam Kher']

In [ ]:
net2[net2.country.str.contains('Israel')].title.unique()

In [ ]:
net2[net2.country.str.contains('Israel')].groupby('actors').title.count().idxmax()

In [ ]:
net2[net2.actors == ' Igal Naor']

Now let's try to count the most prolific actors by country

In [ ]:
n.country.str.split(", ", expand=True).stack().reset_index(1).drop('level_1', axis=1).rename({0: 'single_country'}, axis=1).single_country.unique()

In [ ]:
ll = list(n.country.str.split(", ", expand=True).stack().reset_index(1).drop('level_1', axis=1).rename({0: 'single_country'}, axis=1).single_country.unique())

In [ ]:
l1 = sorted(ll)

In [ ]:
l1

In [ ]:
len(l1)

In [ ]:
dic = {}
for val in ll:
    dic[val] = net2[net2.country.str.contains(val)].groupby('actors').title.count().idxmax()

In [ ]:
dic

# **Indication of viewers' preferences change?**

In [ ]:
countr = n.country.str.split(", ", expand=True).stack().reset_index(1).drop('level_1', axis=1).rename({0: 'single_country'}, axis=1).copy()

In [ ]:
net3 = n.merge(countr, right_index = True, left_index=True).copy()

In [ ]:
net3

In [ ]:
nn = net3.groupby(['release_year', 'single_country']).title.apply(lambda titles: len(titles.tolist())) / net3.groupby('release_year').title.count()

In [ ]:
nn

In [ ]:
nn_US = net3[net3.single_country.str.contains("United States")].groupby('release_year').title.apply(lambda titles: len(titles.tolist())) / net3.groupby('release_year').title.count()

In [ ]:
nn_US.plot.bar(figsize = (9, 7))

In [ ]:
nn_nonUS = net3[~net3.single_country.str.contains('United States')].groupby(['release_year', 'single_country']).title.apply(lambda titles: len(titles.tolist())) / net3.groupby('release_year').title.count()

# ****Let's keep countries that represent a significant number of new entries on a given year

In [ ]:
nn_nonUS[nn_nonUS>0.05].unstack().plot.bar(figsize = (10, 10), stacked = True)

In [ ]:
n[n.release_year == 2017]

# Most represented genres by country****

In [ ]:
genre = n.listed_in.str.split(", ", expand = True).stack().reset_index(1).drop('level_1', axis=1).rename({0:'unique_genre'}, axis=1)
net4 = n.merge(genre, right_index = True, left_index=True).copy()
net4

In [ ]:
dic = {}
for val in l1:
    dic[val] = net4[net4.country.str.contains(val)].groupby('unique_genre').title.count().idxmax()
print(dic)

In [ ]:
print("The most represented genre respectively for {}, {} and {} is respectively {}, {} and {}".format('Unites States', 'France', 'Japan', dic['United States'], dic['France'], dic['Japan']))

# Most produced genres per year (with significant yearly contribution of added titles)****

In [ ]:
nnn = net4.groupby(['release_year', 'unique_genre']).title.count() / net4.groupby('release_year').title.count()#.unstack()#.plot.bar()

In [ ]:
nnn[nnn>0.01].unstack().plot.bar(figsize = (10, 10), stacked = True)

# What is the average time for a movie to reach Netflix catalogue?
****

In [ ]:
n["delta_time"] = n.date_added.dt.year - n.release_year
n.head()

In [ ]:
n.delta_time.mean()